In [2]:
import numpy as np
import pandas as pd
# import skimage.io as skimio
# import skimage.color as skimcolor
# import matplotlib.pyplot as plt
from PIL import Image
import os
import warnings

#### Read in data and modify data

In [4]:
local_path = os.getcwd()
local_path

'C:\\Users\\danny\\Repos\\text_recognition\\preprocess'

In [ ]:
local_path = os.getcwd()
img_dir = "./data/Images/"
trans_dir = "./data/Transcriptions/"
part_dir = "./data/Partitions/"

In [ ]:
# get partitions (only use training for now)
with open(os.path.join(part_dir, "TrainLines.lst")) as f:
    training = f.read().splitlines()
with open(os.path.join(part_dir, "ValidationLines.lst")) as f:
    validation = f.read().splitlines()
with open(os.path.join(part_dir, "TestLines.lst")) as f:
    test = f.read().splitlines()

In [ ]:
# filenames
filenames = [f.replace(".txt", "") for f in os.listdir(trans_dir)]
filenames = [f for f in filenames if f in training]
data_df = pd.DataFrame({"filenames": filenames})
data_df["imgnames"] = [img_dir+f+".png" for f in data_df.filenames]
data_df["transnames"] = [trans_dir+f+".txt" for f in data_df.filenames]

In [ ]:
# images
# def readBenthamImg(fn):
#     im = skimio.imread(fn)
#     rep_val = max(np.median(im[:,:,0]), np.mean(im[:,:,0]))
#     im[im[:,:,3] == 0] = [rep_val, rep_val, rep_val, 255]
#     im = skimcolor.rgb2gray(im)*255
#     return im.astype("int")
def readBenthamImg(fn):
    im = np.array(Image.open(fn))
    rep_val = max(np.median(im[:,:,0]), np.mean(im[:,:,0]))
    im[im[:,:,3] == 0] = [rep_val, rep_val, rep_val, 255]
    im = Image.fromarray(im).convert("L")
    return im
data_df["images"] = [readBenthamImg(f) for f in data_df.imgnames]

In [ ]:
data_df["imsizes"] = [np.array(i).shape for i in data_df.images]
data_df["heights"] = [i[0] for i in data_df.imsizes]
data_df["widths"] = [i[1] for i in data_df.imsizes]

In [ ]:
# transcriptions
def readBenthamTrans(fn):
    return open(fn, "r").readline().replace("\n", "")
data_df["transcription"] = [readBenthamTrans(f) for f in data_df.transnames]

#### Get rid of unwanted rows

In [ ]:
w95 = np.percentile(data_df.widths, 95)
h95 = np.percentile(data_df.heights, 95)
print(w95, h95)

# size columns
# plt.subplot(211)
# plt.hist(data_df.widths)
# plt.axvline(x=w95, color="r")
# plt.subplot(212)
# plt.hist(data_df.heights)
# plt.axvline(x=h95, color="r")
# plt.show()

In [ ]:
# get rid of the really big images
data_df = data_df[np.logical_and(data_df.widths < w95, data_df.heights < h95)]

# get rid of images with annoying characters
data_df["nonhex"] = [any([ord(i) > 127 for i in t]) 
                     for t in data_df.transcription]
data_df = data_df[np.logical_not(data_df.nonhex)]

#### Save new images

In [ ]:
new_img_dir = local_path + "/data/Images_mod/"
if not os.path.isdir(new_img_dir):
    os.mkdir(new_img_dir)
data_df["new_img_path"] = [new_img_dir + f+".png" for f in data_df.filenames]

In [ ]:
count = 0
for i in data_df.index:
    data_df.loc[i, "images"].save(data_df.loc[i, "new_img_path"])
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         skimio.imsave(data_df.loc[i, "new_img_path"], data_df.loc[i, "images"])
    count += 1
    if count % 500 == 0: print(count, end="\t")

#### Save training file

In [ ]:
export_df = data_df[["new_img_path", "transcription"]]
export_df.to_csv("./data/train.csv", sep="\t", index=False)

#### Find freqency of letters

In [ ]:
letters = dict()

for tran in export_df.transcription:
    for l in list(tran):
        if l not in letters:
            letters[l] = 0
        letters[l] += 1
letters = sorted(letters.items(), key = lambda f: f[1], reverse=True)

In [ ]:
print(letters)